In [38]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

from fancyimpute import MICE
import statsmodels.formula.api as smf


#read data set into a pandas dataframe
df = pd.read_csv('~/Documents/Repository/Capstone-1_WorldBank_GenderData/Data.csv')

In [20]:
df.head(2)

,Country.Name,Country.Code,Indicator.Name,Indicator.Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Arab World,ARB,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,3.585854,5.611509,7.350393,8.645366,10.059527,12.358258,15.225028,17.751386,21.102336,NaN
1,Arab World,ARB,"Access to anti-retroviral drugs, male (%)",SH.HIV.ARTC.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.407830,6.393077,7.520210,9.119101,10.627401,11.652603,13.841550,15.956337,18.406402,NaN


In [21]:
# Melting the individual years columns to be one column with the individual years as rows
id_vars = ["Country.Name",
           "Country.Code",
           "Indicator.Name",
          "Indicator.Code"]

dfmelt = pd.melt(frame=df,id_vars=id_vars, var_name="year", value_name="value")

# Formatting 
dfmelt["year"] = dfmelt["year"].str.extract('(\d+)', expand=False).astype(int)
dfmelt["value"] = dfmelt["value"].astype(float)

# Cleaning out unnecessary rows with empty values
dfmelt = dfmelt.dropna()

dfmelt = dfmelt[["Country.Name",
         "Country.Code",
         "Indicator.Name",
         "Indicator.Code",
         "year",
         "value"]]

dfmelt = dfmelt.sort_values(ascending=True, by=["Country.Name","Indicator.Name","year"])

In [22]:
dfmelt.head()

,Country.Name,Country.Code,Indicator.Name,Indicator.Code,year,value
7269408,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2000,0.0
7450352,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2001,0.0
7631296,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2002,0.0
7812240,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2003,0.0
7993184,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2004,0.0


In [23]:
#Find out which years have the least # of missing values
dfmelt.year.value_counts().sort_values(ascending =False).head(16)

2011    47985
2014    44370
2010    42365
2012    41820
2000    39529
2009    37999
2013    37871
2005    37647
2007    36995
2006    36756
2008    36748
2004    34423
2002    33683
2003    33428
2001    33347
1999    31201
Name: year, dtype: int64

In [24]:
#restrict the data to years between 1999 and 2015
df2k = dfmelt[(dfmelt.year > 1999) & (dfmelt.year<2015)]
df2k.head()

,Country.Name,Country.Code,Indicator.Name,Indicator.Code,year,value
7269408,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2000,0.0
7450352,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2001,0.0
7631296,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2002,0.0
7812240,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2003,0.0
7993184,Afghanistan,AFG,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,2004,0.0


In [27]:
# Unmelting/unstacking column "Indictor.Name" to make each 'indicator' variable its own row
dfvars = df2k.pivot_table(index=["Country.Name","year"], columns="Indicator.Name", values="value")
dfvars.reset_index(drop=False, inplace=True)

In [28]:
dfvars.head(20)

Indicator.Name,Country.Name,year,"Access to anti-retroviral drugs, female (%)","Access to anti-retroviral drugs, male (%)","Account at a financial institution, female (% age 15+) [ts]","Account at a financial institution, male (% age 15+) [ts]","Adolescent fertility rate (births per 1,000 women ages 15-19)","Age at first marriage, female","Age at first marriage, male",Age dependency ratio (% of working-age population),...,Women who own land both alone and jointly (% of women age 15-49): Q3,Women who own land both alone and jointly (% of women age 15-49): Q4,Women who own land both alone and jointly (% of women age 15-49): Q5 (highest),Women who own land jointly (% of women age 15-49),Women who own land jointly (% of women age 15-49): Q1 (lowest),Women who own land jointly (% of women age 15-49): Q2,Women who own land jointly (% of women age 15-49): Q3,Women who own land jointly (% of women age 15-49): Q4,Women who own land jointly (% of women age 15-49): Q5 (highest),Women who were first married by age 18 (% of women ages 20-24)
0,Afghanistan,2000,0.0,0.0,NaN,NaN,153.8456,NaN,NaN,103.254202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2001,0.0,0.0,NaN,NaN,150.0468,NaN,NaN,102.933042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2002,0.0,0.0,NaN,NaN,146.2480,NaN,NaN,102.217020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2003,0.0,0.0,NaN,NaN,140.4764,NaN,NaN,101.290161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2004,0.0,0.0,NaN,NaN,134.7048,NaN,NaN,100.247559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Afghanistan,2005,0.0,0.0,NaN,NaN,128.9332,NaN,NaN,99.078444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Afghanistan,2006,0.0,0.0,NaN,NaN,123.1616,NaN,NaN,99.574274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Afghanistan,2007,0.0,0.0,NaN,NaN,117.3900,NaN,NaN,100.000371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Afghanistan,2008,0.0,0.0,NaN,NaN,111.4708,15.0,25.3,100.215886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0
9,Afghanistan,2009,1.0,0.0,NaN,NaN,105.5516,NaN,NaN,100.060480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#check to see if the nature of missing values in rokoandom
#count N/A values in each row and group by year and country
nacount = pd.DataFrame()
nacount['country'] = dfvars['Country.Name']
nacount['year'] = dfvars.year
nacount['nans']= dfvars.isnull().sum(axis=1)
nacount.head()

,country,year,nans
0,Afghanistan,2000,582
1,Afghanistan,2001,600
2,Afghanistan,2002,598
3,Afghanistan,2003,585
4,Afghanistan,2004,572


In [39]:
#nacountry = nacount.groupby(['country']).size()

mod = smf.ols(formula='nans ~ country + year', data=nacount)
res = mod.fit()
res.summary()

PatsyError: Error interpreting categorical data: all items must be hashable
    nans ~ country + year
           ^^^^^^^

In [ ]:
filled = MICE().complete(dfvars)

In [ ]:
dfvars.plot(x = 'Contraceptive prevalence, any methods (% of women ages 15-49)', y ='Life expectancy at birth, total (years)', kind = 'scatter')

In [ ]:
dfvars.to_csv('wrangled_data.csv')